In [42]:
#気象庁
#過去7日間の最高気温のデータを取得するコード
#https://www.data.jma.go.jp/stats/data/mdrr/docs/csv_dl_readme.html
#ウェブ上のデータは毎日5:00ごろ更新＋修正は13:00、19:00、翌日1:00ごろの更新で反映
#定期実行

import pandas as pd
import re

#データを取得してファイル名に日付を入れて保存する
def get_previous_data(d):
    url = f'https://www.data.jma.go.jp/obd/stats/data/mdrr/tem_rct/alltable/mxtemsadext0{d}.csv'
    data = pd.read_csv(url, encoding='Shift-JIS')

    #日付
    yyyymmdd = f"{data['観測時刻(年)'][0]}{data.loc[0,['観測時刻(月)','観測時刻(日)']].apply(lambda x: str(x).zfill(2)).sum()}"
    #ファイル名
    
    filename = f"./data-maxtemp/daily-data/jma-maxtemp-{yyyymmdd}.csv"
    #dataにも記載
    data['date'] = pd.to_datetime(yyyymmdd, format='%Y%m%d')
    #csvで出力
    data.to_csv(filename, index=False)
    return data

#実行
#集計（真夏・猛暑日を数える）
data_agg = pd.DataFrame()

###データ取得
for d in range(1, 8): 
    data = get_previous_data(d)
    ##ととのえる
    ##最高気温のデータのカラムを抽出
    ##（1日前＝昨日の~、2日前=一昨日の~、3日以上前=xx日の最高気温(℃)
    rename_dic = {col:'maxtemp' for col in data.columns if '日の最高気温(℃)' in col}
    data = data.rename(columns=rename_dic)
    ##統合
    data_agg = pd.concat([data_agg, data])

###計算
data_agg['over30'] = data_agg.maxtemp >= 30
data_agg['over35'] = data_agg.maxtemp >= 35
data_agg['over40'] = data_agg.maxtemp >= 40
data_agg['total'] = 1
data_agg['null_values'] = data_agg.maxtemp.isna()

#集計df
heat_points_7days = data_agg.groupby(['date'])[['over30','over35','over40','total','null_values']].sum().reset_index()

#過去分と統合
filename = "./data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-ts.csv"
##過去分を取得
heat_points = pd.read_csv(filename)
heat_points = pd.concat([heat_points,heat_points_7days])
##重複削除（新しく入ったほうを残す）
heat_points.date = pd.to_datetime(heat_points.date)
heat_points = heat_points.loc[~heat_points.duplicated(subset=['date'], keep='last')].reset_index(drop=True)
heat_points = heat_points.dropna(subset='date').sort_values(by='date')

##出力
heat_points.to_csv(filename, index=False)

修正で追加するコード

In [44]:
data_agg

,観測所番号,都道府県,地点,国際地点番号,観測時刻(年),観測時刻(月),観測時刻(日),観測時刻(時),観測時刻(分),maxtemp,...,1日の最高気温起時の品質情報,31日の最高気温の品質情報,31日の最高気温起時（時）,31日の最高気温起時（分）,31日の最高気温起時の品質情報,over30,over35,over40,total,null_values
0,11001,北海道宗谷地方,宗谷岬（ソウヤミサキ）,NaN,2023,8,6,NaN,NaN,18.1,...,NaN,NaN,NaN,NaN,NaN,False,False,False,1,False
1,11016,北海道宗谷地方,稚内（ワッカナイ）,47401.0,2023,8,6,NaN,NaN,18.6,...,NaN,NaN,NaN,NaN,NaN,False,False,False,1,False
2,11046,北海道宗谷地方,礼文（レブン）,NaN,2023,8,6,NaN,NaN,18.5,...,NaN,NaN,NaN,NaN,NaN,False,False,False,1,False
3,11061,北海道宗谷地方,声問（コエトイ）,NaN,2023,8,6,NaN,NaN,19.1,...,NaN,NaN,NaN,NaN,NaN,False,False,False,1,False
4,11076,北海道宗谷地方,浜鬼志別（ハマオニシベツ）,NaN,2023,8,6,NaN,NaN,17.7,...,NaN,NaN,NaN,NaN,NaN,False,False,False,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,94062,沖縄県,西表島（イリオモテジマ）,47917.0,2023,7,31,NaN,NaN,33.2,...,NaN,8.0,14.0,3.0,8.0,True,False,False,1,False
911,94081,沖縄県,石垣島（イシガキジマ）,47918.0,2023,7,31,NaN,NaN,34.2,...,NaN,8.0,14.0,1.0,8.0,True,False,False,1,False
912,94086,沖縄県,盛山（モリヤマ）,NaN,2023,7,31,NaN,NaN,33.0,...,NaN,8.0,11.0,23.0,8.0,True,False,False,1,False
913,94101,沖縄県,大原（オオハラ）,NaN,2023,7,31,NaN,NaN,32.6,...,NaN,8.0,14.0,23.0,8.0,True,False,False,1,False


In [43]:
heat_points

,date,over30,over35,over40,total,null_values
0,2023-07-22,577,25,0,915,0
1,2023-07-23,652,38,0,915,0
2,2023-07-24,714,88,0,915,0
3,2023-07-25,778,167,0,915,0
4,2023-07-26,807,226,0,915,1
5,2023-07-27,804,251,0,915,1
6,2023-07-28,835,233,0,915,0
7,2023-07-29,847,217,0,915,1
8,2023-07-30,775,223,0,915,1
10,2023-07-31,653,124,0,915,2


In [ ]:
#県庁所在地の最高温度をつけたす
#capitol = data_agg[data_agg.capitol==1].set_index(['date','year','pref']).maxtemp.rename('maxtemp_capitol')
#df_count = pd.concat([df_count, capitol],axis=1)